## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,2022-02-12 15:11:19,19.7297,-155.0900,67.73,84,100,1.01,overcast clouds
1,1,Punta Arenas,CL,2022-02-12 15:06:58,-53.1500,-70.9167,57.31,47,40,36.82,scattered clouds
2,2,Sofiyivka,UA,2022-02-12 15:11:19,48.0493,33.8746,35.98,71,100,18.75,overcast clouds
3,3,Qaanaaq,GL,2022-02-12 15:11:19,77.4840,-69.3632,-21.21,48,0,8.55,clear sky
4,4,Georgetown,MY,2022-02-12 15:11:11,5.4112,100.3354,83.89,74,20,1.99,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,2022-02-12 15:11:19,19.7297,-155.0900,67.73,84,100,1.01,overcast clouds
4,4,Georgetown,MY,2022-02-12 15:11:11,5.4112,100.3354,83.89,74,20,1.99,few clouds
5,5,Lorengau,PG,2022-02-12 15:11:20,-2.0226,147.2712,80.80,75,92,3.33,overcast clouds
7,7,Port Alfred,ZA,2022-02-12 15:11:20,-33.5906,26.8910,80.51,78,9,18.75,clear sky
9,9,Naze,JP,2022-02-12 15:11:21,28.3667,129.4833,62.06,89,100,14.43,overcast clouds
11,11,Cabo San Lucas,MX,2022-02-12 15:08:03,22.8909,-109.9124,66.51,49,40,6.91,scattered clouds
14,14,Panama City,PA,2022-02-12 15:09:08,8.9936,-79.5197,89.17,58,13,5.99,few clouds
16,16,Hithadhoo,MV,2022-02-12 15:11:21,-0.6000,73.0833,81.93,66,76,8.75,broken clouds
17,17,Concepcion,CL,2022-02-12 15:11:22,-36.8270,-73.0498,80.40,34,0,9.22,clear sky
18,18,Barbate,ES,2022-02-12 15:11:23,36.1924,-5.9219,68.09,57,93,12.39,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                339
City                   339
Country                336
Date                   339
Lat                    339
Lng                    339
Max Temp               339
Humidity               339
Cloudiness             339
Wind Speed             339
Current Description    339
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,67.73,overcast clouds,19.7297,-155.0900,
4,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
5,Lorengau,PG,80.80,overcast clouds,-2.0226,147.2712,
7,Port Alfred,ZA,80.51,clear sky,-33.5906,26.8910,
9,Naze,JP,62.06,overcast clouds,28.3667,129.4833,
11,Cabo San Lucas,MX,66.51,scattered clouds,22.8909,-109.9124,
14,Panama City,PA,89.17,few clouds,8.9936,-79.5197,
16,Hithadhoo,MV,81.93,broken clouds,-0.6000,73.0833,
17,Concepcion,CL,80.40,clear sky,-36.8270,-73.0498,
18,Barbate,ES,68.09,overcast clouds,36.1924,-5.9219,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,67.73,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
4,Georgetown,MY,83.89,few clouds,5.4112,100.3354,Cititel Penang
5,Lorengau,PG,80.80,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
7,Port Alfred,ZA,80.51,clear sky,-33.5906,26.8910,The Halyards Hotel
9,Naze,JP,62.06,overcast clouds,28.3667,129.4833,Shishime Hotel
...,...,...,...,...,...,...,...
658,Quelimane,MZ,78.75,broken clouds,-17.8786,36.8883,Hotel Flamingo
659,Ponta Delgada,PT,61.52,broken clouds,37.7333,-25.6667,Vila Nova Hotel
663,Bartica,GY,77.32,overcast clouds,6.4000,-58.6167,Sloth Island Nature Resort
664,Mayumba,GA,82.09,few clouds,-3.4320,10.6554,LIKWALE LODGE


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                 weights=[max(temp, 0) for temp in max_temp],
                 dissipating=False,
                 max_intensity=300, 
                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))